In [67]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

# TavilySearchResults 검색 엔진
from langchain_community.tools.tavily_search import TavilySearchResults

#Output response
from langchain_teddynote.messages import stream_response

# API 설정
from dotenv import load_dotenv


In [68]:
# API 키 세팅
load_dotenv()

True

# General LLM

In [69]:
question = "2024 KBO MVP에 대해 알려줘"

In [70]:
# 모델 4o 이용
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

print(llm.invoke(question).content)

2024 KBO 리그 MVP에 대한 정보는 현재로서는 제공할 수 없습니다. KBO 리그의 MVP는 시즌이 끝난 후 성적과 활약을 바탕으로 선정되기 때문에, 시즌이 진행 중이거나 아직 시작되지 않은 경우에는 예측하기 어렵습니다. 시즌이 진행되면서 선수들의 성적과 팀의 성과를 지켜보는 것이 중요합니다. 시즌이 끝난 후에는 공식 발표를 통해 MVP가 누구인지 알 수 있을 것입니다.


# RAG implementation

In [71]:
# Question
question = "2024 KBO MVP에 대해 알려줘"

In [72]:
# 검색 결과 인스턴스를 생성
search = TavilySearchResults(k=10)
# 검색 결과를 가져오기
output = search.invoke(question)
text = ""
for tmp in output :
    text += " " + tmp['content'] + "출처 :" + tmp['url'] + "\n"

In [73]:
# 텍스트 분리 및 임베딩
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_text(text)

embedding_function = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(splits, embedding=embedding_function)

# 포함되어있는 정보 검색
retriever = vectorstore.as_retriever()

# 프롬프트 템플릿 이용
prompt = PromptTemplate.from_template(
"""당신은 정보 검색을 전문적으로 수행하는 AI 어시스턴트입니다. 
사용자가 질문한 내용을 정확하고 신뢰할 수 있도록 제공된 문맥(context)을 기반으로 답변을 생성해야 합니다.

**지침**
1. 주어진 문맥(context)에서만 답변하세요.
2. 문맥에 관련된 정보가 없다면, 답을 추측하지 말고 "주어진 정보에서 질문에 대한 내용을 찾을 수 없습니다."라고 응답하세요.
3. 응답을 명확하고 간결하게 정리하세요.
4. 기술 용어나 고유 명사는 번역하지 말고 그대로 유지하세요.
5. 가능하면 원문의 주요 문장을 요약하여 활용하세요.
6. 검색에 참고한 URL도 함께 출력하세요.

**입력 데이터**
- 질문: 사용자의 질문
- 문맥: 검색된 문서에서 제공된 정보

**출력 형식**
- **답변**: 질문에 대한 직접적인 응답
- **출처**: 답변에 참고한 URL

---

🔹 **질문:**  
{question}

🔹 **문맥:**  
{context}

---

✅ **답변:**"""
)

# 모델 4o 이용
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 체인 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)



In [74]:
answer = rag_chain.stream(question)
stream_response(answer)

**답변:** 2024 KBO 리그에서 KIA 타이거즈의 김도영이 MVP로 선정되었습니다. 그는 38홈런과 40도루를 기록하며 뛰어난 성적을 거두었습니다.

**출처:** [조선일보](https://www.chosun.com/sports/baseball/2024/11/26/KHS7BOWC2VE7TPOSXKWIPQSGNM/), [국제뉴스](https://www.gukjenews.com/news/articleView.html?idxno=3144980)